# Week 3: Identify Risk Factors for Infection

<span style="color:red">
**UPDATE**

Thank you again for the previous analysis. We will next be publishing a public health advisory that warns of specific infection risk factors of which individuals should be aware. Please advise as to which population characteristics are associated with higher infection rates. 
</span>

Your goal for this notebook will be to identify key potential demographic and economic risk factors for infection by comparing the infected and uninfected populations.

## Imports

In [1]:
import cudf
import cuml
import cupy as cp

## Load Data

Begin by loading the data you've received about week 3 of the outbreak into a cuDF data frame. The data is located at `./data/week3.csv`. For this notebook you will need all columns of the data.

In [2]:
gdf = cudf.read_csv('./data/week3.csv')
gdf.head()

 missing cuda symbols while dynamic loading
 cuFile initialization failed


,age,sex,employment,infected
0,0,m,U,0.0
1,0,m,U,0.0
2,0,m,U,0.0
3,0,m,U,0.0
4,0,m,U,0.0


## Calculate Infection Rates by Employment Code

Convert the `infected` column to type `float32`. For people who are not infected, the float32 `infected` value should be `0.0`, and for infected people it should be `1.0`.

In [3]:
gdf['infected'].astype('float32')

0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
           ... 
58479889    0.0
58479890    0.0
58479891    0.0
58479892    0.0
58479893    0.0
Name: infected, Length: 58479894, dtype: float32

Now, produce a list of employment types and their associated **rates** of infection, sorted from highest to lowest rate of infection.

**NOTE**: The infection **rate** for each employment type should be the percentage of total individuals within an employment type who are infected. Therefore, if employment type "X" has 1000 people, and 10 of them are infected, the infection **rate** would be .01. If employment type "Z" has 10,000 people, and 50 of them are infected, the infection rate would be .005, and would be **lower** than for type "X", even though more people within that employment type were infected.

In [4]:
infected = gdf[['infected', 'employment']].groupby('employment').mean()
infected.columns = ['rates']
infected = infected.sort_values('rates', ascending=False)
infected

,rates
employment,
Q,0.012756
I,0.010354
V,0.007590
P,0.006190
Z,0.005655
"R, S, T",0.005390
O,0.005284
L,0.004970
G,0.004948


Finally, read in the employment codes guide from `./data/code_guide.csv` to interpret which employment types are seeing the highest rates of infection.

In [5]:
guide = cudf.read_csv('./data/code_guide.csv')
guide

,Code,Field
0,A,"Agriculture, forestry & fishing"
1,"B, D, E","Mining, energy and water supply"
2,C,Manufacturing
3,F,Construction
4,G,"Wholesale, retail & repair of motor vehicles"
5,H,Transport & storage
6,I,Accommodation & food services
7,J,Information & communication
8,K,Financial & insurance activities
9,L,Real estate activities


In [6]:
infected = infected.merge(guide, left_on='employment', right_on='Code')
infected.sort_values('rates', ascending=False)
infected

,rates,Code,Field
16,0.000217,U,Student
17,0.007590,V,Retired
18,0.004539,X,Outside the UK or not specified
20,0.005655,Z,Not formally employed
0,0.003853,A,"Agriculture, forestry & fishing"
1,0.003774,"B, D, E","Mining, energy and water supply"
2,0.003882,C,Manufacturing
3,0.003182,F,Construction
4,0.004948,G,"Wholesale, retail & repair of motor vehicles"
5,0.003388,H,Transport & storage


## Calculate Infection Rates by Employment Code and Sex

We want to see if there is an effect of `sex` on infection rate, either in addition to `employment` or confounding it. Group by both `employment` and `sex` simultaneously to get the infection rate for the intersection of those categories.

In [7]:
infected = gdf[['infected', 'employment', 'sex']].groupby(['sex', 'employment']).mean()
infected

infected
sex employment          
f   M           0.007645
    A           0.007491
    Q           0.014947
    U           0.000329
    Z           0.007629
m   C           0.002569
    H           0.002482
    U           0.000110
f   V           0.010852
    K           0.007672
    C           0.007630
m   V           0.003685
f   P           0.007584
m   L           0.002197
f   N           0.007389
    H           0.007385
m   I           0.005117
    N           0.002538
    B, D, E     0.002462
    Q           0.005120
f   X           0.007391
m   F           0.002535
    A           0.002514
    K           0.002490
    X           0.002435
f   J           0.007645
    L           0.007221
    B, D, E     0.007973
m   P           0.002577
    J           0.002546
    Z           0.002543
    G           0.002596
    M           0.002520
f   F           0.007577
m   R, S, T     0.002542
f   O           0.007719
m   O           0.002543
f   G           0.007556
    R, S, T     0.007748
    I           0.015064

In [9]:
infected.columns = ['rates']
infected = infected.sort_values('rates', ascending=False).reset_index()
infected = infected.merge(guide, left_on='employment', right_on='Code')
infected

,sex,employment,rates,Code,Field
0,f,I,0.015064,I,Accommodation & food services
1,f,Q,0.014947,Q,Human health & social work activities
2,f,V,0.010852,V,Retired
3,f,"B, D, E",0.007973,"B, D, E","Mining, energy and water supply"
4,f,"R, S, T",0.007748,"R, S, T",Other services
5,f,O,0.007719,O,Public admin & defence; social security
6,f,K,0.007672,K,Financial & insurance activities
7,f,M,0.007645,M,"Professional, scientific & technical activities"
8,f,J,0.007645,J,Information & communication
9,f,C,0.007630,C,Manufacturing


## Take the Assessment

After completing the work above, visit the *Launch Section* web page that you used to launch this Jupyter Lab. Scroll down below where you launched Jupyter Lab, and answer the question *Week 3 Assessment*. You can view your overall progress in the assessment by visiting the same *Launch Section* page and clicking on the link to the *Progress* page. On the *Progress* page, if you have successfully answered all the assessment questions, you can click on *Generate Certificate* to receive your certificate in the course.

![launch_task_page](./images/launch_task_page.png)

<div align="center"><h2>Optional: Restart the Kernel</h2></div>

If you plan to continue work in other notebooks, please shutdown the kernel.

In [10]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}